## Pre-processing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

filename1 = 'drive/MyDrive/TextAnalytics/training_set_1.csv'
filename2 = 'drive/MyDrive/TextAnalytics/training_set_2.csv'
filename3 = 'drive/MyDrive/TextAnalytics/test_set.csv'
filename4 = 'drive/MyDrive/TextAnalytics/remaining_set.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV

In [3]:
df_test = pd.read_csv(filename3)
df_train_remaining = pd.read_csv(filename4)

df_train_remaining.head(2)

,Unnamed: 0,review_rating,date,year,review_title,body,product_title,product_rating,language,english,pos_review,body_tok,body_tok_ngrams,body_lem_ngrams
0,31164,5,"November 18, 2017",2017,Super phone,A fabulous phone. Complaints that you see on l...,Samsung Galaxy S8 Plus (S8+) (SM-G955FD) 4GB R...,4.0,en,1,1,"['fabulous', 'phone', 'complaints', 'see', 'li...","['fabulous', 'phone', 'complaints', 'see', 'li...","['LEM_fabulous', 'LEM_phone', 'LEM_complaint',..."
1,31166,5,"July 25, 2017",2017,Five Stars,As advertised. Brand new. Fast shipping.,Samsung Galaxy S8 Plus (S8+) (SM-G955FD) 4GB R...,4.0,en,1,1,"['advertised', 'brand', 'new', 'fast', 'shippi...","['advertised', 'brand', 'new', 'fast', 'shippi...","['LEM_advertise', 'LEM_brand', 'LEM_new', 'LEM..."


In [4]:
df_train_remaining = df_train_remaining.sort_values(by='year', ascending=True) # Ordine decrescente

df_train_remaining = df_train_remaining[0:10000]

df_train_remaining.head(2)

,Unnamed: 0,review_rating,date,year,review_title,body,product_title,product_rating,language,english,pos_review,body_tok,body_tok_ngrams,body_lem_ngrams
13336,36,1,"September 23, 2009",2009,Worst Experience with Amazon,This is probably the wrost experience i had wi...,"Samsung Alias2 U750 Phone, Black (Verizon Wire...",3.0,en,1,0,"['probably', 'wrost', 'experience', 'amazon', ...","['probably', 'wrost', 'experience', 'amazon', ...","['LEM_probably', 'LEM_wrost', 'LEM_experience'..."
13331,2,5,"December 13, 2009",2009,Excellent product,The product has been very good. I had used thi...,Motorola I265 phone,3.0,en,1,1,"['product', 'good', 'used', 'cell', 'phone', '...","['product', 'good', 'used', 'cell', 'phone', '...","['LEM_product', 'LEM_good', 'LEM_use', 'LEM_ce..."


In [5]:
# Selezione X e Y

train = df_train_remaining[['body', 'review_rating']]

test = df_test[['body', 'review_rating']]

train.head(4)

,body,review_rating
13336,This is probably the wrost experience i had wi...,1
13331,The product has been very good. I had used thi...,5
13332,For the price you really get a solid boost mot...,5
13335,I got this phone just as secondary cell phone....,3


In [6]:
# Preparazione training set
x_train1 = train['body']
y_train1 = train['review_rating']

# Preparazione test set
x_test = test['body']
y_test = test['review_rating']

len(x_train1),len(y_train1), len(x_test),len(y_test)

(10000, 10000, 20000, 20000)

In [7]:
# Funzione per metriche di Evaluation

def model_evaluation(real_v, pred_v):
    print(f"Accuracy score: {accuracy_score(real_v, pred_v)}")
    print("Classification report:")
    print(classification_report(real_v, pred_v))
    cm = confusion_matrix(real_v, pred_v)
    print (f"Confusion matrix \n {cm}")

## SVM con TF-IDF su Remaining Set

In [8]:
# Preprocessing
vectorizer = TfidfVectorizer(min_df=5,ngram_range=(1, 3)) # scelta n degli n-grams
vectorizer.fit(x_train1)

# Training
training1_features = vectorizer.transform(x_train1)
test_features = vectorizer.transform(x_test)
learner = SVC()
classifier1 = learner.fit(training1_features, y_train1)

# Predizioni
predictions1 = classifier1.predict(test_features)

# Evaluation
model_evaluation(y_test, predictions1)

Accuracy score: 0.69855
Classification report:
              precision    recall  f1-score   support

           1       0.68      0.74      0.71      3709
           2       0.00      0.00      0.00      1018
           3       0.31      0.02      0.04      1348
           4       0.39      0.06      0.11      2518
           5       0.71      0.97      0.82     11407

    accuracy                           0.70     20000
   macro avg       0.42      0.36      0.33     20000
weighted avg       0.60      0.70      0.62     20000

Confusion matrix 
 [[ 2734     2     9    14   950]
 [  521     0     5    22   470]
 [  403     1    26    83   835]
 [  172     0    22   155  2169]
 [  209     0    23   119 11056]]
